# Installation


In [ ]:
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl

--2024-05-06 10:30:37--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl.1’

layoutparser-0.0.0- 100%[===================>]  18.26M  4.92MB/s    in 16s     

2024-05-06 10:30:55 (1.14 MB/s) - ‘layoutparser-0.0.0-py3-none-any.whl.1’ saved [19145360/19145360]



In [ ]:
!pip install -U layoutparser-0.0.0-py3-none-any.whl

Processing ./layoutparser-0.0.0-py3-none-any.whl
layoutparser is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
!pip install paddleocr

In [ ]:
!pip install layoutparser-0.0.0-py3-none-any.whl

Processing ./layoutparser-0.0.0-py3-none-any.whl
layoutparser is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
!pip install paddlepaddle

# Conversion

In [ ]:
from pdf2image import convert_from_path

In [ ]:
images = convert_from_path('aetech_efi300623.pdf')

In [ ]:
mkdir pages

In [ ]:
for i in range(len(images)):
  images[i].save('pages/pages'+str(i)+'.jpg','JPEG')

# Layout Extraction

In [ ]:
import cv2
import layoutparser as lp
image = cv2.imread("pages/pages1.jpg")
image = image[..., ::-1]
#load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                      threshold=0.5,
                                      label_map={0: "Text", 1:"Title", 2:"List", 3:"Table", 4:"Figure"},
                                      enforce_cpu=False,
                                      enable_mkldnn=True
                                      )
#detect
layout = model.detect(image)

ModuleNotFoundError: No module named 'paddle'

In [ ]:
layout

NameError: name 'layout' is not defined

In [ ]:
x_1=213.68804931640625
y_1=443.869873046875

x_2=1427.833740234375
y_2=1509.9595947265625

In [ ]:
#select only the tables
for l in layout :
  if l.type == 'Table':
   x_1 = int(l.block.x_1)
   y_1 = int(l.block.y_1)
   x_2 = int(l.block.x_2)
   y_2 = int(l.block.y_2)

  break

In [ ]:
print(x_1,y_1,x_2,y_2)

In [ ]:
im = cv2.imread('pages/pages1.jpg')

In [ ]:
cv2.imwrite('ext_im.jpg',im[y_1:y_2, x_1:x_2])

# Text Detection and Recongnition

In [ ]:
from paddleocr import PaddleOCR, draw_ocr

In [ ]:
ocr = PaddleOCR(lang="fr")
image_path="ext_im.jpg"
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]

In [ ]:
print(image_height, image_width)

In [ ]:
output = ocr.ocr(image_path)

In [ ]:
print(output)

In [ ]:
output[0]

In [ ]:
image_boxes = image_cv.copy()

In [ ]:
boxes = []
boxes_texts = []
boxes_texts_probs = []
for line in output:
    for word_info in line:
        bbox = word_info[0]
        text = word_info[1][0]
        prob = word_info[1][1]
        boxes_texts_probs.append((bbox, text, prob))
        boxes_texts.append(text)
# Print the extracted bounding box coordinates and text

for text in boxes_texts :
  print("text:",text)

In [ ]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]


# Reconstruction

In [ ]:
im = image_cv.copy()

In [ ]:
import cv2

horiz_boxes = []
vert_boxes = []
image_height, image_width, _ = im.shape  # Assuming 'im' is the image you're working with

for box_info in boxes_texts_probs:
    bbox = box_info[0]
    x_h, x_v = 0, int(bbox[0][0])
    y_h, y_v = int(bbox[0][1]), 0
    width_h, width_v = image_width, int(bbox[2][0] - bbox[0][0])
    height_h, height_v = int(bbox[2][1] - bbox[0][1]), image_height

    horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
    vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])

    cv2.rectangle(im, (x_h, y_h), (x_h + width_h, y_h + height_h), (255, 0, 0), 1)
    cv2.rectangle(im, (x_v, y_v), (x_v + width_v, y_v + height_v), (0, 255, 0), 1)


In [ ]:
boxes_texts_probs[0]

In [ ]:
cv2.imwrite('horiz_vert.jpg',im)

In [ ]:
horiz_boxes[0]

In [ ]:
vert_boxes[0]

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

# Generate a list of probabilities from boxes_texts_probs
prob_list = [box_info[2] for box_info in boxes_texts_probs]

# Perform non-maximum suppression
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    prob_list,
    max_output_size=1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)


In [ ]:
import numpy as np
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

In [ ]:
# Check if boxes has at least four elements before accessing boxes[3]
if len(boxes) >= 4:
    print("Element at index 3:", boxes[3])
else:
    print("The boxes list does not have enough elements to access index 3.")


In [ ]:
im_nms = image_cv.copy()

In [ ]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [ ]:
cv2.imwrite('im_nms.jpg',im_nms)

In [ ]:
import tensorflow as tf

# Generate a list of probabilities from boxes_texts_probs
prob_list = [box_info[2] for box_info in boxes_texts_probs]

# Perform non-maximum suppression
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    prob_list,
    max_output_size=1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)


In [ ]:
import numpy as np
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

In [ ]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)


In [ ]:
cv2.imwrite('im_nms.jpg',im_nms)

In [ ]:
out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

In [ ]:
unordered_boxes = []
for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

In [ ]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

In [ ]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [ ]:
texts = [line[1][0] for line in output]


In [ ]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [ ]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )
    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]
    print(resultant)


In [ ]:
import re

for i in range(len(horiz_lines)):
    for j in range(len(vert_lines)):
        resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]])

        matched_texts = []
        for b in range(len(boxes)):
            the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
            if iou(resultant, the_box) > 0.1:
                matched_texts.append(texts[b])

        # Vérifier si la première lettre de chaque mot est en majuscule
        if matched_texts:
            first_text = matched_texts[0]
            if re.match(r'^[A-Z]', first_text):
                combined_text = first_text
                for k in range(1, len(matched_texts)):
                    if re.match(r'^[a-z]', matched_texts[k]):
                        combined_text += " " + matched_texts[k]
                out_array[i][j] = combined_text

In [ ]:
out_array=np.array(out_array)

In [ ]:
horiz_boxes

In [ ]:
out_array

In [ ]:
pd.DataFrame(out_array).to_csv('sample1.csv')